# Establish a Baseline Training and Validation Pipeline

## Initial Setup

In [ ]:
!pip install -U ultralytics wandb

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import yaml

from ultralytics import YOLO

import wandb
from wandb.integration.ultralytics import add_wandb_callback

/opt/conda/envs/odsc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



## Initialize a WandB Run

In [3]:
run = wandb.init(project="mlops-odsc", entity="ml-colabs", job_type="trial")

wandb: Currently logged in as: ayush-thakur (ml-colabs). Use `wandb login --relogin` to force relogin


## Fetch and preprocess the dataset

We first fetch the bdd-100k dataset hosted as a W&B dataset artifact.

In [ ]:
artifact = wandb.use_artifact(
    "reviewco/object-detection-bdd/bdd100k-ultralytics-format:latest", type="dataset"
)
artifact_dir = artifact.download()

Next, we preprocess the `data.yaml` file in the artifact, to set the absolute path to the dataset. This file would be used by Ultralytics to register the dataset and build the dataloader for training and validation.

In [5]:
artifact_dir = "data"

In [6]:
metadata_file = os.path.join(artifact_dir, "data.yaml")
# with open(metadata_file, "r") as yaml_file:
#     metadata = yaml.safe_load(yaml_file)
# metadata["path"] = os.path.join(
#     os.path.dirname(os.path.abspath("")), artifact_dir
# )
# with open(metadata_file, "w") as yaml_file:
#     yaml.dump(metadata, yaml_file)

## Training and Validation

In [7]:
# Define the YOLO model and load the respective pre-trained checkpoint
model = YOLO("yolov8n.pt")

# Add the wandb callback for ultralytics, this would enable us to
# utilize several advanced visualization features.
# add_wandb_callback(model, enable_model_checkpointing=True)

In [8]:
# Train the model
model.train(data=metadata_file, epochs=5, imgsz=640)

# Validate the model
model.val()

# Finish the experiment
# wandb.finish()

New https://pypi.org/project/ultralytics/8.0.200 available 😃 Update with 'pip install -U ultralytics'
/opt/conda/envs/odsc/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data/data.yaml, epochs=5, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verb

RuntimeError: Dataset 'data/data.yaml' error ❌ 
Dataset 'data/data.yaml' images not found ⚠️, missing path '/home/ayushthakur/odsc/datasets/odsc-yolo-e2e/data/valid/images'
Note dataset download directory is '/home/ayushthakur/odsc/datasets'. You can update this in '/home/ayushthakur/.config/Ultralytics/settings.yaml'